# Домашнее задание №1. Инструмент для анализа текста

**Основная задача**: Создайте инструмент для анализа текста, который обрабатывает текстовые записи (например, отзывы пользователей, статьи или отрывки из книг) и предоставляет некоторые выводы, статистику и преобразования. После этого примените инструмент для анализа текста в соответствии с вашим вариантом и сделайте выводы о характере текста.

## Определение анализируемого текста

In [4]:
import textwrap

surname = "Шегай"  # Ваша фамилия

if not surname:
    raise Exception('Необходимо указать фамилию!')

alp = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
w = [1, 4, 21, 25, 34,  6, 44, 26, 13, 44, 38, 26, 4, 43,  4, 49, 46,
        17, 42, 29,  4,  9, 36, 34, 31, 22,  15, 30,  4, 19, 28, 28, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 4 + 1

print("Вариант: ", variant)

# Construct the file name based on the variant number
file_name = f"data/{variant}.txt"

# Read the contents of the file
with open(file_name, "r", encoding="utf-8") as file:
    TEXT = file.read()

wrapped = textwrap.fill(TEXT[:500], width=80)
print(f"Анализируемый текст: \n{wrapped}...")

Вариант:  1
Анализируемый текст: 
**_Stellar Blade_** is the latest exclusive to hit the PlayStation 5 that excels
at delivering a highly riveting action RPG. Shift Up Corporation has crafted an
impressive linear title that takes influences from some of the most renowned
games in recent times. **Even though the parallels are quite apparent, _Stellar
Blade_ doesn’t try to reinvent the wheel but rather refines what players are
already familiar with.**  _Stellar Blade_ is centered around EVE, a member of
the 7th Airborne Division w...


# Задача 1: Сбор и предварительная обработка данных

**Задание:**

Написать функцию `preprocess_text`, которая принимает на вход текст и очищает его. Функция должна:
* Преобразовывать текст в строчные буквы.
* Удалять ненужные знаки препинания и специальные символы, но сохранять знаки препинания в конце предложения (. ! ?).
* Удалить лишние пробелы между словами.
* Вернуть очищенный текст.

**Решение:**

In [8]:
def preprocess_text(text: str) -> str:
    import re
    
    ttext = text.lower()
    ttext = re.sub(r'[^a-z\s.!?0-9\']', '', ttext)
    ttext = re.sub(r'([.!?])\1+', r'\1', ttext)
    ttext = re.sub(r'\s+', ' ', ttext).strip()

    return ttext


**Описание решения:**

Функция `preprocess_text` использует библиотеку `re` и обрабатывает заданную строку при помощи регулярных выражений. Функция работает по следующему алгоритму:

1) При помощи метода `lower()` преобразовывает текст в строчные буквы.
2) При помощи метода `sub` из библиотеки `re` удаляет все символы, кроме латинских букв и знаков (. ! ?).
3) При помощи метода `sub` из библиотеки `re` удаляет все повторяющиеся знаки препинания.
4) При помощи метода `sub` из библиотеки `re` удаляет все лишние пробелы.

# Задача 2: Анализ частоты слов

**Задание:**

Написать функцию `word_frequency`, которая берет очищенный текст (из задания 1) и возвращает словарь, где ключами являются слова, а значениями - их частота в тексте. Функция должна:
* Подсчитать, как часто каждое слово встречается в тексте.
* Игнорировать любые общие стоп-слова. Вы можете использовать предопределенный список, приведенный ниже, или использовать свой собственный. Подумайте о том, чтобы сделать его настраиваемым.

**Предопределённые стоп-слова:**

In [9]:
stop_words = ["i", "and", "the", "is", "in", "it", "you", "that", "to", "of", "a", "with", "for", "on", "this", "at", "by", "an", "its"]

**Решение:**

In [10]:
def word_frequency(text: str, stop_words=None) -> dict:
    import re
    stop_words = stop_words if stop_words else ["i", "and", "the", "is", "in", "it", "you", "that", "to", 
                                                "of", "a", "with", "for", "on", "this", "at", "by", "an", "its"]
    words = dict()
    ttext = preprocess_text(text)
    ttext = re.sub(r'[^a-z\s\']', '', ttext).split()
    
    for word in ttext:
        if word in stop_words:
            pass
        elif word not in words:
            words[word] = 1
        else:
            words[word] += 1

    return words


**Описание решения:**

Функция `word_frequency` работает по следующему алгоритму:

1) При помощи метода `sub` из библиотеки `re` удаляет все знаки препинания.
2) При помощи метода `split()` разделяет полученный текст на массив из слов
3) Функция перебирает каждое слово в массиве и проверяет, находится ли оно в списке стоп-слов и есть ли оно в словаре.
4) Если слово есть в списке стоп-слов, функция его пропускает
5) Если слова нет в словаре, в словарь добавляется новый элемент с ключом, соответсвующим самому слову, и значением `1`.
6) Если слово есть в словаре, функция находит в нем элемент с соответсвтующим ключом и добавляет к значению +1.

# Задача 3: Извлечение информации

**Задача:** 

Написать функцию `extract_information`, которая берет неочищенный текст (не из задания 1) и извлекает определенные типы информации на основе настраиваемых шаблонов `regex`, предоставленных в качестве keyword-аргументов. Функция должна возвращать словарь, в котором ключи - это типы совпадений, а значения - списки найденных совпадений.

**Поддерживаемые типы информации:**

* Адреса электронных почт
* Телефонные номера
* Даты
* Время
* Цены
* Дополнительные данные по желанию пользователя

**Решение:**

In [11]:
def extract_information(text: str, *, 
                        email_pattern=r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
                        phone_pattern=r'[\+\(]\d[\d\s\-\)]{8,12}\d', 
                        date_pattern=r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b',
                        time_pattern=r'\b\d{1,2}:\d{2}\s*[APap][mM]?\b', 
                        price_pattern=r'\$\d+(?:\.\d{2})?',
                        **extra_patterns) -> dict:
    import re

    information = {
        'emails': re.findall(email_pattern, text),
        'phone_numbers': re.findall(phone_pattern, text),
        'dates': re.findall(date_pattern, text),
        'times': re.findall(time_pattern, text),
        'prices': re.findall(price_pattern, text)
    }

    for key, pattern in extra_patterns.items():
        information[key] = re.findall(pattern, text)
        
    return information


**Описание решения:**

Функция `extract_information` работает по следующему алгоритму:

1) Создается исходный словарь `information`
2) При помощи метода `findall` из библиотеки `re` находит все совпадения с паттернами по умолчанию и записывает их в словарь.
3) При помощи цикла `for` поочередно перебирает паттерны дополнительных данных.
4) При помощи метода `findall` из библиотеки `re` находит все совпадения с дополнительными паттернами и записывает их в словарь.

# Задача 4: Анализ настроения

**Задание:**

Написать функцию `analyze_sentiment`, которая берет очищенный текст (из задания 1) и анализирует его настроение на основе заранее определенных положительных и отрицательных слов. Функция должна возвращать оценку настроения текста.

**Предназначенные списки слов:**

* Положительные слова (по умолчанию):

In [12]:
positive_words = ["good", "great", "happy", "joy", "excellent", "fantastic", "love", "best", "amazing", "fun"]

* Негативные слова (по умлочанию):

In [13]:
negative_words = ["bad", "sad", "hate", "terrible", "awful", "poor", "worst"]

**Решение:**

In [14]:
def analyze_sentiment(text: str, positive_words=None, negative_words=None) -> int:
    import re
    positive_words = positive_words if positive_words else ["good", "great", "happy", "joy", "excellent", "fantastic", 
                                                            "love", "best", "amazing", "fun"]
    negative_words = negative_words if negative_words else ["bad", "sad", "hate", "terrible", "awful", "poor", "worst"]
    sentiment_score = 0
    ttext = preprocess_text(text)
    ttext = re.sub(r'[^a-z\s]', '', preprocess_text(ttext)).split()
    for word in ttext:
        if word in positive_words:
            sentiment_score += 1
        elif word in negative_words:
            sentiment_score -= 1
        else: pass
    
    return sentiment_score


**Описание решения:**

Функция `analyze_sentiment` работает по следующему алгоритму:

1) При помощи метода `sub` из библиотеки `re` удаляет из текста все символы, кроме букв и пробелов.
2) При помощи метода `split` разделяет полученную строку на массив слов.
3) Поочередно рассматривает каждое слово в массиве. 
4) Если встречается слово из массива `positive_words`, к промежуточной сумме `sentimet_score` добавляется 1.
5) Если встречается слово из массива `negative_words`, из промежуточной сумме `sentimet_score` вычитается 1.

# Задача 5: Обобщение текста

**Задание:**

Написать функцию `summarize_text`, которая берет очищенный текст (из задания 1) и обобщает его, извлекая наиболее важные предложения на основе их релевантности. Функция должна позволять пользователю указывать в качестве параметра желаемый коэффициент сжатия.

**Решение:**

In [15]:
def summarize_text(text: str, compression_ratio: float, min_threshold: int=2):
    import re
    ttext = text
    sentences = re.split(r'(?<=\w[.!?])\s+', ttext)
    if len(sentences) <= max(len(sentences) * compression_ratio, min_threshold):
        
        return ttext

    def sentence_importance(sentence: str) -> int:
        return analyze_sentiment(sentence) + len(sentence.split())

    importances = [[sentence, sentence_importance(sentence)] for sentence in sentences]

    sorted_importances = sorted(importances, key=lambda x: x[1])
    sentences.remove(sorted_importances[0][0])

    return summarize_text(' '.join(sentences), compression_ratio, min_threshold)

**Описание решения:**

Функция `summarize_text` работает по следующему алгоритму:

1) При помощи метода `split` из библиотеки `re` преобразуем исходный текст в массив предложений.
2) Сравниваем текущую длину массива с максимумом среди `len(sentences) * compression_ratio` и `min_threshold`.
3) Вложенная функция `sentence_importance` служит для определения важности предложения.
4) Создается массив из предложений и значений их важности.
5) Массив сортируется по важности от наименее важного к наиболее важному.
6) Из оригинального массива предложений удаляется наименее важное предложение.
7) Функция вызывается до тех пор, пока длина массива предложений не окажется меньше или равна максимуму среди `len(sentences) * compression_ratio` и `min_threshold`.

# Задание 6: Визуализация частоты слов

**Задание:**

Написать функцию `visualize_word_frequency`, которая берет словарь частот слов (полученный в задании 2) и визуализирует частоты в текстовом формате.

**Решение:**

In [16]:
def visualize_word_frequency(word_frequencies: dict, max_threshold=20):
    sorted_word_frequencies = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)
    for index, element in enumerate(sorted_word_frequencies):
        if index < max_threshold:
            print(f'{element[0]}:'.ljust(max(map(len, word_frequencies))+1), element[1]*"*")


**Объяснение решения:**

Функция `visualize_word_frequency` работает по следующему алгоритму:

1) Входной словарь `word_frequencies` сортируется по возрастанию значений элементов и преобразуется в массив.
2) Поочередно рассматривается каждый элемент полученного массива.
3) Если индекс элемента не превышает значение `max_threshold` (по умолчанию 20), на экран выводится слово и символ `*`, число которого соответсвует значению из словаря.

# Задание 7: Функции высшего порядка для анализа текста

**Задача:**

Написать функцию `apply_analysis`, которая принимает список функций анализа и одну текстовую запись. Функция должна применять каждую функцию анализа к текстовой записи и возвращать словарь результатов.

**Решение:**

In [17]:
def apply_analysis(text: str, functions) -> dict:
    results = dict()
    ttext = text
    for function in functions:
        results[function.__name__] = function(ttext)

    return results


**Объяснение решения:**

Функция `apply_analysis` работает по следующему алгоритму:

1) Создается пустой словарь `results`.
2) Поочередно рассматривается каждая функция из входного массива функций.
3) В словарь записывается результат выполнения очередной функции из массива. В качестве ключа служит имя функции.

# Задача 8: Обернуть всё в класс

**Задание:**

Создать класс `TextAnalyzer`, который инкапсулирует всю функциональность из предыдущих задач. Класс должен включать методы для каждого из следующих действий:

1) Инициализация:

 * Класс должен принимать один текст при инициализации, хранить исходный текст и автоматически создавать его предобработанную версию.
2) Методы:

* `word_frequency`: Анализирует частоту слов из предварительно обработанного текста.
* `extract_information`: Извлекает электронные письма, номера телефонов, даты, время и цены из предварительно обработанного текста.
* `analyze_sentiment`: Выполняет анализ настроения предварительно обработанного текста.
* `summarize_text`: Обобщает предварительно обработанный текст на основе коэффициента сжатия.
* `visualize_word_frequency`: Визуализирует данные о частоте слов в тексте.
* `apply_analysis`: Применяет список функций анализа к предварительно обработанному тексту.

**Решение:**

In [26]:
import re


def word_frequency(text: str, stop_words=None) -> dict:

    stop_words = stop_words if stop_words else ["i", "and", "the", "is", "in", "it", "you", "that", "to", 
                                                "of", "a", "with", "for", "on", "this", "at", "by", "an", "its"]
    words = dict()
    ttext = preprocess_text(text)
    ttext = re.sub(r'[^a-z\s\']', '', ttext).split()
    
    for word in ttext:
        if word in stop_words:
            pass
        elif word not in words:
            words[word] = 1
        else:
            words[word] += 1

    return words

def analyze_sentiment(text: str, positive_words=None, negative_words=None) -> int:
    import re
    positive_words = positive_words if positive_words else ["good", "great", "happy", "joy", "excellent", "fantastic", 
                                                            "love", "best", "amazing", "fun"]
    negative_words = negative_words if negative_words else ["bad", "sad", "hate", "terrible", "awful", "poor", "worst"]
    sentiment_score = 0
    ttext = re.sub(r'[^a-z\s]', '', preprocess_text(text)).split()
    for word in ttext:
        if word in positive_words:
            sentiment_score += 1
        elif word in negative_words:
            sentiment_score -= 1
        else: pass
    
    return sentiment_score

class TextAnalyzer:

    def __init__(self, text: str):
        self.original_text = text
        self.cleaned_text = self.preprocess_text(text)

    def preprocess_text(self, text: str) -> str:
    
        ttext = text.lower()
        ttext = re.sub(r'[^a-z\s.!?0-9\'@+]', '', ttext)
        ttext = re.sub(r'([.!?])\1+', r'\1', ttext)
        ttext = re.sub(r'\s+', ' ', ttext).strip()

        return ttext

    def word_frequency(self, censored=None) -> dict:

        censored = censored if censored else ["and", "the", "is", "in", "it", "you", "that", "to", "of", "a", "with",
                                              "for", "on", "this", "at", "by", "an"]
        words = dict()
        ttext = re.sub(r'[^a-z\s\']', '', self.cleaned_text).split()

        for word in ttext:
            if word in censored:
                pass
            elif word not in words:
                words[word] = 1
            else:
                words[word] += 1

        return words

    def extract_information(self, *,
                            email_pattern=r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
                            phone_pattern=r'\+\d[\d -]{8,12}\d',
                            date_pattern=r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b',
                            time_pattern=r'\b\d{1,2}:\d{2}\s*[APap][mM]?\b',
                            price_pattern=r'\$\d+(?:\.\d{2})?',
                            **extra_patterns) -> dict:

        information = {
            'emails': re.findall(email_pattern, self.cleaned_text),
            'phone_numbers': re.findall(phone_pattern, self.cleaned_text),
            'dates': re.findall(date_pattern, self.cleaned_text),
            'times': re.findall(time_pattern, self.cleaned_text),
            'prices': re.findall(price_pattern, self.cleaned_text)
        }

        for key, pattern in extra_patterns.items():
            information[key] = re.findall(pattern, self.cleaned_text)

        return information

    def analyze_sentiment(self, positive_words=None, negative_words=None) -> int:
        positive_words = positive_words if positive_words else ["good", "great", "happy", "joy", "excellent", "fantastic",
                                                                "love", "best", "amazing", "fun"]
        negative_words = negative_words if negative_words else ["bad", "sad", "hate", "terrible", "awful", "poor",
                                                                "worst"]
        sentiment_score = 0
        for word in self.cleaned_text.split():
            if word in positive_words:
                sentiment_score += 1
            elif word in negative_words:
                sentiment_score -= 1
            else:
                pass

        return sentiment_score

    def summarize_text(self, compression_ratio: float, min_threshold=2):

        sentences = re.split(r'(?<=\w[.!?])\s+', self.cleaned_text)
        if len(sentences) <= max(len(sentences) * compression_ratio, min_threshold):
            return self.cleaned_text

        def sentence_importance(sentence: str) -> int:
            s = TextAnalyzer(sentence)
            return s.analyze_sentiment() + len(sentence.split())

        importances = [[sentence, sentence_importance(sentence)] for sentence in sentences]

        sorted_importances = sorted(importances, key=lambda x: x[1])
        sentences.remove(sorted_importances[0][0])
        s = TextAnalyzer(' '.join(sentences))
        return s.summarize_text(compression_ratio, min_threshold)

    def visualize_word_frequency(self, max_threshold=20):

        sorted_word_frequencies = sorted(self.word_frequency().items(), key=lambda item: item[1], reverse=True)
        for index, element in enumerate(sorted_word_frequencies):
            if index < max_threshold:
                print(f'{element[0]}:'.ljust(max(map(len, self.word_frequency())) + 1), element[1] * "*")

    def apply_analysis(self, functions: list) -> dict:

        results = dict()
        
        for function in functions:
            results[function.__name__] = function(self.cleaned_text)

        return results


# Задача 9: Анализ текста

**Задание:**

Провести анализ предложенного соответствии с вариантом текста. Необходимо:

* Нормализовать текст
* Узнать наиболее часто встречающиеся слова
* Оценить настроение текста
* Определить основную информацию текста

**Решение:**

In [27]:
ryan_gosling = TextAnalyzer(TEXT)

print(f'Нормализованный текст:\n{ryan_gosling.cleaned_text}\n')
print(f'Анализ частоты слов:\n{ryan_gosling.word_frequency()}\n')
print(f'Настроение текста: {ryan_gosling.analyze_sentiment()}\n')
print(f'Основная информация текста:\n{ryan_gosling.summarize_text(0.6)}\n')

Нормализованный текст:
stellar blade is the latest exclusive to hit the playstation 5 that excels at delivering a highly riveting action rpg. shift up corporation has crafted an impressive linear title that takes influences from some of the most renowned games in recent times. even though the parallels are quite apparent stellar blade doesnt try to reinvent the wheel but rather refines what players are already familiar with. stellar blade is centered around eve a member of the 7th airborne division who has come back to a ravaged earth from an offworld colony. she along with other warriors has been sent to investigate the origins of the naytibas and reclaim the planet for humanity. during the course of her mission eve discovers that the history between the naytibas and humans is far more complicated than anticipated. stellar blade stellar blade is an immediate mustplay for ps5. pros spectacular combat mechanics solid performance cool abilities evocative soundtrack cons inconsistent narr

**Вывод о содержании текста:**

Текст представляет собой обзор игры Stellar Blade. Игра была разработана компанией Shift Up Corporation и сочетает в себе элементы из других игр, но не пытается "изобрести велосипед", а совершенствует уже знакомые механики. 

Наиболее часто встречающимися словами являются "stellar" и "blade".

Анализ настроения текста показал положительное число, что позволяет сделать вывод о том, что отзыв положительный.

Основная информация текста заключается в следующих тезисах:
* В игре потрясающая боевая механика и отличная производительность
* В игре представлены классные способности и запоминающийся саундтрек
* Повествование ведется в непоследовательном темпе, а также есть небольшие визуальные проблемы
* Боевой геймплей - это выдающаяся особенность с веселыми и волнующими столкновениями в полуоткрытом мире
* Игра имеет сходство с Nier Automata и Jedi Fallen Order
* Дальнейшее построение мира могло бы пойти на пользу вселенной, но в ней есть повороты, интересные последовательности действий и то, что нужно игрокам в action RPG

**Вывод по лабораторной работе:**

Были изучены основы работы с текстом в языке программирования Python. Был создан инструмент для анализа текста, который обрабатывает текстовые записи (например, отзывы пользователей, статьи или отрывки из книг) и предоставляет некоторые выводы, статистику и преобразования. Инструмент был применен для анализа текста в соответсвии с 25м варантом, сделаны выводы о характере текста.